This notebook contains tests and validations that are easier checked manually.

TODO: Eventually, this should be replaced by fuzzy statistical tests.

### Centralised training

Verify client.train by running a centralised training loop.

In [ ]:
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from flwr.common import Context

from fedppo.client_app import client_fn
from fedppo.main import FedPPOConfig, CONFIG_DIR

config = FedPPOConfig.from_raw(OmegaConf.load(CONFIG_DIR + "/cartpole.yaml"))

client = client_fn(
    context=Context(0, 0, {}, {}, {}),
    task_cfg=config.task,
    client_cfg=config.client,
).client

metrics = client.train(parameters=client.loss.state_dict(), config={})[2]

num_metrics = len(metrics)
num_cols = 3
num_rows = (num_metrics + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
axes = axes.flatten()

for idx, (key, values) in enumerate(metrics.items()):
    axes[idx].plot(values)
    axes[idx].set_title(key)
    axes[idx].set_xlabel("Iterations")
    axes[idx].set_ylabel(key.split("/")[-1])
    axes[idx].grid(True)

# Hide any empty subplots
for i in range(idx + 1, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()